# <center>Freshwater Transport by Eddies within the Bay of Bengal's Central Axis</center>
<center>Alex Herron<center>
<center>January 4th, 2022</center>

# Contents
0. [Downloading Data](#downloading_data)
1. [Formatting Data](#formatting_data)
2. [Preliminary Figures](#preliminary_figures)
3. [Calculating Slopes for Each Cross-section](#calculating_slopes)
4. [Converting Slopes to Sverdrups](#convert_to_sv)
5. [Figure 3 (Scatter/line for both 10/15 Degree Bands)](#figure_3)
6. [Multiple Linear Regression](#mlr)
7. [Various Horizontal Mixing Coefficients](#various_kh)

# 0. Downloading Data <a name="downloading_data"></a>

In [1]:
#Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from mpl_toolkits.mplot3d import Axes3D

In [2]:
#Create list of file names for each available file
f_2003 = "BoB_band.2003.sub.dat"
f_2004 = "BoB_band.2004.sub.dat"
f_2005 = "BoB_band.2005.sub.dat"
f_2006 = "BoB_band.2006.sub.dat"
f_2007 = "BoB_band.2007.sub.dat"
f_2008 = "BoB_band.2008.sub.dat"
f_2009 = "BoB_band.2009.sub.dat"
f_2010 = "BoB_band.2010.sub.dat"
f_2011 = "BoB_band.2011.sub.dat"
f_2012 = "BoB_band.2012.sub.dat"
f_2013 = "BoB_band.2013.sub.dat"
f_2014 = "BoB_band.2014.sub.dat"
f_2015 = "BoB_band.2015.sub.dat"
f_2016 = "BoB_band.2016.sub.dat"
f_2017 = "BoB_band.2017.sub.dat"
f_2018 = "BoB_band.2018.sub.dat"

file_list = [f_2003, f_2004, f_2005, f_2006, f_2007, f_2008, f_2009, f_2010, f_2011, f_2012,
            f_2013, f_2014, f_2015, f_2016, f_2017, f_2018]

In [3]:
#Create empty dataframe with column names
column_names = ['depth', 'salinity', 'latitude', 'longitude', 'timestamp']
df = pd.DataFrame(columns = column_names)

In [4]:
#Initialize frames (for concatenating dataframes from multiple files)
frames = []

#Loop through each file, extract depth, salinity, latitude, longitude, and timestamp
for file in file_list:    
    with open(file) as datFile:
        dep = np.array([float(data.split()[0]) for data in datFile])
    with open(file) as datFile:
        sal = np.array([float(data.split()[2]) for data in datFile])    
    with open(file) as datFile:
        lat = np.array([float(data.split()[5]) for data in datFile])
    with open(file) as datFile:
        lon = np.array([float(data.split()[6]) for data in datFile])
    with open(file) as datFile:
        ts = np.array([data.split()[8] for data in datFile])
    
    #Create dataframe for each file/year
    year = file[9:13]
    y = year
    
    #Use dictionary for making dataframe
    data = {'depth': dep, 
            'salinity': sal,
            'latitude': lat,
            'longitude': lon, 
            'timestamp': ts, 
            'year': y
           }
    
    #Concatenate data frames to make complete dataframe
    year = file[9:13]
    df_i = pd.DataFrame(data)
    frames.append(df_i)
    df = pd.concat(frames)

In [5]:
#Need to change indexes for consistency 
df.index = (range(0, len(df)))
df

,depth,salinity,latitude,longitude,timestamp,year
0,5.3,34.042,12.911,86.946,2900093.0223.20031219,2003
1,9.2,34.043,12.911,86.946,2900093.0223.20031219,2003
2,19.2,34.045,12.911,86.946,2900093.0223.20031219,2003
3,29.4,34.055,12.911,86.946,2900093.0223.20031219,2003
4,39.5,34.055,12.911,86.946,2900093.0223.20031219,2003
...,...,...,...,...,...,...
1119511,488.0,35.125,13.337,87.077,6901741.1845.20180302,2018
1119512,513.0,35.122,13.337,87.077,6901741.1845.20180302,2018
1119513,538.0,35.119,13.337,87.077,6901741.1845.20180302,2018
1119514,563.0,35.115,13.337,87.077,6901741.1845.20180302,2018


# 1. Formatting Data <a name="formatting_data"></a>

In [6]:
#Create column for months
month = []
for i in range(0, len(df)):
    month.append(df['timestamp'][i][17:19])
    
df['month'] = month

In [7]:
#Create column for days
day = []
for i in range(0, len(df)):
    day.append(df['timestamp'][i][19:21])
    
df['day'] = day

In [ ]:
#Create column for date
date = []
for i in range(0, len(df)):
    date_i = str(df['year'][0]) + "-" + str(df['month'][0]) + "-" + str(df['day'][0])
    date.append(date_i)
df['date'] = date

In [ ]:
#Create column for season
season = []

spring = ['03', '04', '05']
summer = ['06', '07', '08']
fall = ['09', '10', '11']
winter = ['12', '01', '02']

for i in range(0, len(month)):
    if (month[i] in spring):
        season.append('spring')
    elif (month[i] in summer):
        season.append('summer')
    elif (month[i] in fall):
        season.append('fall')
    elif (month[i] in winter):
        season.append('winter')
        
df['season'] = season

In [ ]:
#Create column for freshwater
freshwater = []

for i in range(0, len(df['salinity'])):
    freshwater.append(1000 - df['salinity'][i])
    
df['freshwater'] = freshwater

In [ ]:
#Make dataframe for longitude and latitude for multiple linear regression later on
fw = df['freshwater']
lat_lon = pd.concat([df['latitude'], df['longitude']], axis = 1)

In [ ]:
#Create dataframe for each season
df_winter = df.loc[df['season'] == 'winter']
df_spring = df.loc[df['season'] == 'spring']
df_summer = df.loc[df['season'] == 'summer']
df_fall = df.loc[df['season'] == 'fall']

In [ ]:
#Create dataframe for each depth slab
df_0_50 = df.loc[df['depth'] < 50]
df_50_100 = df.loc[(df['depth'] >= 50) & (df['depth'] < 100)]
df_100_150 = df.loc[(df['depth'] >= 100) & (df['depth'] < 150)]
df_150_200 = df.loc[(df['depth'] >= 150) & (df['depth'] < 200)]
df_200_250 = df.loc[(df['depth'] >= 200) & (df['depth'] < 250)]
df_250_300 = df.loc[(df['depth'] >= 250) & (df['depth'] < 300)]
df_0_300 = df.loc[df['depth'] < 300]

In [ ]:
#Create dataframe for latitude ranges
df_lat10 = df.loc[(df['latitude'] >= 8) & (df['latitude'] <= 12)]
df_lat15 = df.loc[(df['latitude'] >= 13) & (df['latitude'] <= 17)]

# 2. Preliminary Figures <a name="preliminary_figures"></a>

In [ ]:
#Figure 1
plt.scatter(df['longitude'], df['latitude'], s = 0.01, c = 'red')
plt.scatter(df_lat10['longitude'], df_lat10['latitude'], s = 0.01, c = 'blue')
plt.scatter(df_lat15['longitude'], df_lat15['latitude'], s = 0.01, c = 'green')
plt.grid()
plt.title('Distribution of Argo floats 2003 - 2018')
plt.xlabel('Longitude (degrees)')
plt.ylabel('Latitude (degrees)')
plt.legend()
plt.gca().legend(('Bay of Bengal Band', '10 Degree Band', '15 Degree Band'))

In [ ]:
#Figure 2

#Linear regression - predicting y from x
x = df_0_300['latitude'].values.reshape(-1, 1)
y = df_0_300['freshwater'].values.reshape(-1, 1)

linear_regressor = LinearRegression()  # create object for the class
linear_regressor.fit(x, y)  # perform linear regression
y_pred = linear_regressor.predict(x)  # make predictions

#Gather coefficient and intercept for line of best fit
ols = linear_model.LinearRegression()
model = ols.fit(x, y)
coeff = model.coef_
lat_coeff = str(coeff[0][0])[:10]
intercept = str(model.intercept_[0])[:10]
text = "Line of best fit: y = " + lat_coeff + " * latitude + " + intercept

#Create scatterplot with line of best fit
plt.scatter(x, y, s = 0.001)
plt.plot(x, y_pred, color = 'red')
plt.grid()
plt.title('Bay of Bengal Band, Freshwater by Latitude, Annual, 0-300m')
plt.xlabel('Latitude (Degrees North)')
plt.ylabel('Freshwater (parts per thousand)')
plt.text(5, 975, text)
plt.show()

# 3. Calculating Slopes for Each Cross-section <a name="calculating_slopes"></a>

In [ ]:
#General slope computation method
x = df['latitude']
y = df['freshwater']

slope, intercept, r, p, stderr = linregress(x, y)

print(f"Slope: {slope}")
print(f"Intercept: {intercept}")
print(f"R: {r}")
print(f"p-value: {p}")
print(f"Standard Error: {stderr}")

In [ ]:
#Create dataframes for latitude bands split by depth slabs

#Latitude = 10 +/- 2 degree band
df_lat10_0_300 = df_lat10.loc[df_lat10['depth'] < 300]
df_lat10_0_50 = df_lat10.loc[df_lat10['depth'] < 50]
df_lat10_50_100 = df_lat10.loc[(df_lat10['depth'] >= 50) & (df_lat10['depth'] < 100)]
df_lat10_100_150 = df_lat10.loc[(df_lat10['depth'] >= 100) & (df_lat10['depth'] < 150)]
df_lat10_150_200 = df_lat10.loc[(df_lat10['depth'] >= 150) & (df_lat10['depth'] < 200)]
df_lat10_200_250 = df_lat10.loc[(df_lat10['depth'] >= 200) & (df_lat10['depth'] < 250)]
df_lat10_250_300 = df_lat10.loc[(df_lat10['depth'] >= 250) & (df_lat10['depth'] < 300)]

#Latitude = 15 +/- 2 degree band
df_lat15_0_300 = df_lat15.loc[df_lat15['depth'] < 300]
df_lat15_0_50 = df_lat15.loc[df_lat15['depth'] < 50]
df_lat15_50_100 = df_lat15.loc[(df_lat15['depth'] >= 50) & (df_lat15['depth'] < 100)]
df_lat15_100_150 = df_lat15.loc[(df_lat15['depth'] >= 100) & (df_lat15['depth'] < 150)]
df_lat15_150_200 = df_lat15.loc[(df_lat15['depth'] >= 150) & (df_lat15['depth'] < 200)]
df_lat15_200_250 = df_lat15.loc[(df_lat15['depth'] >= 200) & (df_lat15['depth'] < 250)]
df_lat15_250_300 = df_lat15.loc[(df_lat15['depth'] >= 250) & (df_lat15['depth'] < 300)]

## Annual Slopes (North -> South)

In [ ]:
#Create empty dataframe with column names
depth_columns = ['direction', '0-300', '0-50', '50-100', '100-150', '150-200', '200-250', '250-300']
season_rows = ['annual', 'fall', 'winter', 'spring', 'summer']
df_15_s_flux = pd.DataFrame(np.empty((5,8)), columns = depth_columns, index = season_rows)
df_10_s_flux = pd.DataFrame(np.empty((5,8)), columns = depth_columns, index = season_rows)

In [ ]:
#Get slopes for latitude = 10 +/-2 band (annual)
df_lat10_depth = [df_lat10_0_300, df_lat10_0_50, df_lat10_50_100, df_lat10_100_150, df_lat10_150_200,
                 df_lat10_200_250, df_lat10_250_300]

lat10_annual_slopes = []
lat10_data_num = []
for depth_slab in df_lat10_depth:
    df = depth_slab
    x = df['latitude']
    y = df['freshwater']
    slope, intercept, r, p, stderr = linregress(x, y)
    lat10_annual_slopes.append(slope)
    lat10_data_num.append(len(x))
    
df_10_s_flux['direction'][0] = 180
df_10_s_flux['0-300'][0] = lat10_annual_slopes[0]
df_10_s_flux['0-50'][0] = lat10_annual_slopes[1]
df_10_s_flux['50-100'][0] = lat10_annual_slopes[2]
df_10_s_flux['100-150'][0] = lat10_annual_slopes[3]
df_10_s_flux['150-200'][0] = lat10_annual_slopes[4]
df_10_s_flux['200-250'][0] = lat10_annual_slopes[5]
df_10_s_flux['250-300'][0] = lat10_annual_slopes[6]

In [ ]:
#Get slopes for latitude = 15 +/-2 band (annual)
df_lat15_depth = [df_lat15_0_300, df_lat15_0_50, df_lat15_50_100, df_lat15_100_150, df_lat15_150_200,
                 df_lat15_200_250, df_lat15_250_300]

lat15_annual_slopes = []
lat15_data_num = []
for depth_slab in df_lat15_depth:
    df = depth_slab
    x = df['latitude']
    y = df['freshwater']
    slope, intercept, r, p, stderr = linregress(x, y)
    lat15_annual_slopes.append(slope)
    lat15_data_num.append(len(x))
    
df_15_s_flux['direction'][0] = 180
df_15_s_flux['0-300'][0] = lat15_annual_slopes[0]
df_15_s_flux['0-50'][0] = lat15_annual_slopes[1]
df_15_s_flux['50-100'][0] = lat15_annual_slopes[2]
df_15_s_flux['100-150'][0] = lat15_annual_slopes[3]
df_15_s_flux['150-200'][0] = lat15_annual_slopes[4]
df_15_s_flux['200-250'][0] = lat15_annual_slopes[5]
df_15_s_flux['250-300'][0] = lat15_annual_slopes[6]

## Seasonal Slopes (North -> South)

In [ ]:
#Create dataframe for cross-sections of season and depth for 10 degree latitude band
#North-South slope

season_list = ['fall', 'winter', 'spring', 'summer']

for i in range(0, len(season_list)):
    df_lat10_f = df_lat10.loc[df_lat10['season'] == season_list[i]]
    df_lat10_f_0_300 = df_lat10_f.loc[df_lat10_f['depth'] < 300]
    df_lat10_f_0_50 = df_lat10_f.loc[df_lat10_f['depth'] < 50]
    df_lat10_f_50_100 = df_lat10_f.loc[(df_lat10_f['depth'] >= 50) & (df_lat10_f['depth'] < 100)]
    df_lat10_f_100_150 = df_lat10_f.loc[(df_lat10_f['depth'] >= 100) & (df_lat10_f['depth'] < 150)]
    df_lat10_f_150_200 = df_lat10_f.loc[(df_lat10_f['depth'] >= 150) & (df_lat10_f['depth'] < 200)]
    df_lat10_f_200_250 = df_lat10_f.loc[(df_lat10_f['depth'] >= 200) & (df_lat10_f['depth'] < 250)]
    df_lat10_f_250_300 = df_lat10_f.loc[(df_lat10_f['depth'] >= 250) & (df_lat10_f['depth'] < 300)]

    #Get slopes for fall latitude = 10 +/-2 band (annual)
    df_lat10_f_depth = [df_lat10_f_0_300, df_lat10_f_0_50, df_lat10_f_50_100, df_lat10_f_100_150, df_lat10_f_150_200,
                     df_lat10_f_200_250, df_lat10_f_250_300]

    lat10_f_slopes = []
    lat10_f_data_num = []

    df_lat10_f_depth[0]
    for depth_slab in df_lat10_f_depth:
        df = depth_slab
        x = df['latitude']
        y = df['freshwater']
        slope, intercept, r, p, stderr = linregress(x, y)
        lat10_f_slopes.append(slope)
        lat10_f_data_num.append(len(x))

    df_10_s_flux['direction'][i+1] = 180
    df_10_s_flux['0-300'][i+1] = lat10_f_slopes[0]
    df_10_s_flux['0-50'][i+1] = lat10_f_slopes[1]
    df_10_s_flux['50-100'][i+1] = lat10_f_slopes[2]
    df_10_s_flux['100-150'][i+1] = lat10_f_slopes[3]
    df_10_s_flux['150-200'][i+1] = lat10_f_slopes[4]
    df_10_s_flux['200-250'][i+1] = lat10_f_slopes[5]
    df_10_s_flux['250-300'][i+1] = lat10_f_slopes[6]

In [ ]:
#Create dataframe for cross-sections of season and depth for 15 degree latitude band
#North-South slope

season_list = ['fall', 'winter', 'spring', 'summer']

for i in range(0, len(season_list)):
    df_lat15_i = df_lat15.loc[df_lat15['season'] == season_list[i]]
    df_lat15_i_0_300 = df_lat15_i.loc[df_lat15_i['depth'] < 300]
    df_lat15_i_0_50 = df_lat15_i.loc[df_lat15_i['depth'] < 50]
    df_lat15_i_50_100 = df_lat15_i.loc[(df_lat15_i['depth'] >= 50) & (df_lat15_i['depth'] < 100)]
    df_lat15_i_100_150 = df_lat15_i.loc[(df_lat15_i['depth'] >= 100) & (df_lat15_i['depth'] < 150)]
    df_lat15_i_150_200 = df_lat15_i.loc[(df_lat15_i['depth'] >= 150) & (df_lat15_i['depth'] < 200)]
    df_lat15_i_200_250 = df_lat15_i.loc[(df_lat15_i['depth'] >= 200) & (df_lat15_i['depth'] < 250)]
    df_lat15_i_250_300 = df_lat15_i.loc[(df_lat15_i['depth'] >= 250) & (df_lat15_i['depth'] < 300)]

    #Get seasonal slopes for 15 degree band
    df_lat15_i_depth = [df_lat15_i_0_300, df_lat15_i_0_50, df_lat15_i_50_100, df_lat15_i_100_150, df_lat15_i_150_200,
                     df_lat15_i_200_250, df_lat15_i_250_300]

    lat15_i_slopes = []
    lat15_i_data_num = []

    df_lat15_i_depth[0]
    for depth_slab in df_lat15_i_depth:
        df = depth_slab
        x = df['latitude']
        y = df['freshwater']
        slope, intercept, r, p, stderr = linregress(x, y)
        lat15_i_slopes.append(slope)
        lat15_i_data_num.append(len(x))

    df_15_s_flux['direction'][i+1] = 180
    df_15_s_flux['0-300'][i+1] = lat15_i_slopes[0]
    df_15_s_flux['0-50'][i+1] = lat15_i_slopes[1]
    df_15_s_flux['50-100'][i+1] = lat15_i_slopes[2]
    df_15_s_flux['100-150'][i+1] = lat15_i_slopes[3]
    df_15_s_flux['150-200'][i+1] = lat15_i_slopes[4]
    df_15_s_flux['200-250'][i+1] = lat15_i_slopes[5]
    df_15_s_flux['250-300'][i+1] = lat15_i_slopes[6]
    
df_15_s_flux

## Annual Slopes (East -> West)

In [ ]:
#Create empty dataframe with column names
df_15_w_flux = pd.DataFrame(np.empty((5,8)), columns = depth_columns, index = season_rows)
df_10_w_flux = pd.DataFrame(np.empty((5,8)), columns = depth_columns, index = season_rows)

In [ ]:
#Get slopes for latitude = 10 +/-2 band (annual)
df_lat10_depth = [df_lat10_0_300, df_lat10_0_50, df_lat10_50_100, df_lat10_100_150, df_lat10_150_200,
                 df_lat10_200_250, df_lat10_250_300]

lat10_annual_slopes = []
lat10_data_num = []
for depth_slab in df_lat10_depth:
    df = depth_slab
    x = df['longitude']
    y = df['freshwater']
    slope, intercept, r, p, stderr = linregress(x, y)
    lat10_annual_slopes.append(slope)
    lat10_data_num.append(len(x))
    
df_10_w_flux['direction'][0] = 270
df_10_w_flux['0-300'][0] = lat10_annual_slopes[0]
df_10_w_flux['0-50'][0] = lat10_annual_slopes[1]
df_10_w_flux['50-100'][0] = lat10_annual_slopes[2]
df_10_w_flux['100-150'][0] = lat10_annual_slopes[3]
df_10_w_flux['150-200'][0] = lat10_annual_slopes[4]
df_10_w_flux['200-250'][0] = lat10_annual_slopes[5]
df_10_w_flux['250-300'][0] = lat10_annual_slopes[6]

In [ ]:
#Get slopes for latitude = 15 +/-2 band (annual)
df_lat15_depth = [df_lat15_0_300, df_lat15_0_50, df_lat15_50_100, df_lat15_100_150, df_lat15_150_200,
                 df_lat15_200_250, df_lat15_250_300]

lat15_annual_slopes = []
lat15_data_num = []
for depth_slab in df_lat15_depth:
    df = depth_slab
    x = df['longitude']
    y = df['freshwater']
    slope, intercept, r, p, stderr = linregress(x, y)
    lat15_annual_slopes.append(slope)
    lat15_data_num.append(len(x))
    
df_15_w_flux['direction'][0] = 270
df_15_w_flux['0-300'][0] = lat15_annual_slopes[0]
df_15_w_flux['0-50'][0] = lat15_annual_slopes[1]
df_15_w_flux['50-100'][0] = lat15_annual_slopes[2]
df_15_w_flux['100-150'][0] = lat15_annual_slopes[3]
df_15_w_flux['150-200'][0] = lat15_annual_slopes[4]
df_15_w_flux['200-250'][0] = lat15_annual_slopes[5]
df_15_w_flux['250-300'][0] = lat15_annual_slopes[6]

## Seasonal Slopes (East -> West)

In [ ]:
#Create dataframe for cross-sections of season and depth for 15 degree latitude band
#East-West slope

season_list = ['fall', 'winter', 'spring', 'summer']

for i in range(0, len(season_list)):
    df_lat10_f = df_lat10.loc[df_lat10['season'] == season_list[i]]
    df_lat10_f_0_300 = df_lat10_f.loc[df_lat10_f['depth'] < 300]
    df_lat10_f_0_50 = df_lat10_f.loc[df_lat10_f['depth'] < 50]
    df_lat10_f_50_100 = df_lat10_f.loc[(df_lat10_f['depth'] >= 50) & (df_lat10_f['depth'] < 100)]
    df_lat10_f_100_150 = df_lat10_f.loc[(df_lat10_f['depth'] >= 100) & (df_lat10_f['depth'] < 150)]
    df_lat10_f_150_200 = df_lat10_f.loc[(df_lat10_f['depth'] >= 150) & (df_lat10_f['depth'] < 200)]
    df_lat10_f_200_250 = df_lat10_f.loc[(df_lat10_f['depth'] >= 200) & (df_lat10_f['depth'] < 250)]
    df_lat10_f_250_300 = df_lat10_f.loc[(df_lat10_f['depth'] >= 250) & (df_lat10_f['depth'] < 300)]

    #Get slopes for fall latitude = 10 +/-2 band (annual)
    df_lat10_f_depth = [df_lat10_f_0_300, df_lat10_f_0_50, df_lat10_f_50_100, df_lat10_f_100_150, df_lat10_f_150_200,
                     df_lat10_f_200_250, df_lat10_f_250_300]

    lat10_f_slopes = []
    lat10_f_data_num = []

    df_lat10_f_depth[0]
    for depth_slab in df_lat10_f_depth:
        df = depth_slab
        x = df['longitude']
        y = df['freshwater']
        slope, intercept, r, p, stderr = linregress(x, y)
        lat10_f_slopes.append(slope)
        lat10_f_data_num.append(len(x))

    df_10_w_flux['direction'][i+1] = 270
    df_10_w_flux['0-300'][i+1] = lat10_f_slopes[0]
    df_10_w_flux['0-50'][i+1] = lat10_f_slopes[1]
    df_10_w_flux['50-100'][i+1] = lat10_f_slopes[2]
    df_10_w_flux['100-150'][i+1] = lat10_f_slopes[3]
    df_10_w_flux['150-200'][i+1] = lat10_f_slopes[4]
    df_10_w_flux['200-250'][i+1] = lat10_f_slopes[5]
    df_10_w_flux['250-300'][i+1] = lat10_f_slopes[6]

In [ ]:
season_list = ['fall', 'winter', 'spring', 'summer']

for i in range(0, len(season_list)):
    df_lat15_i = df_lat15.loc[df_lat15['season'] == season_list[i]]
    df_lat15_i_0_300 = df_lat15_i.loc[df_lat15_i['depth'] < 300]
    df_lat15_i_0_50 = df_lat15_i.loc[df_lat15_i['depth'] < 50]
    df_lat15_i_50_100 = df_lat15_i.loc[(df_lat15_i['depth'] >= 50) & (df_lat15_i['depth'] < 100)]
    df_lat15_i_100_150 = df_lat15_i.loc[(df_lat15_i['depth'] >= 100) & (df_lat15_i['depth'] < 150)]
    df_lat15_i_150_200 = df_lat15_i.loc[(df_lat15_i['depth'] >= 150) & (df_lat15_i['depth'] < 200)]
    df_lat15_i_200_250 = df_lat15_i.loc[(df_lat15_i['depth'] >= 200) & (df_lat15_i['depth'] < 250)]
    df_lat15_i_250_300 = df_lat15_i.loc[(df_lat15_i['depth'] >= 250) & (df_lat15_i['depth'] < 300)]

    #Get seasonal slopes for 15 degree band
    df_lat15_i_depth = [df_lat15_i_0_300, df_lat15_i_0_50, df_lat15_i_50_100, df_lat15_i_100_150, df_lat15_i_150_200,
                     df_lat15_i_200_250, df_lat15_i_250_300]

    lat15_i_slopes = []
    lat15_i_data_num = []

    df_lat15_i_depth[0]
    for depth_slab in df_lat15_i_depth:
        df = depth_slab
        x = df['longitude']
        y = df['freshwater']
        slope, intercept, r, p, stderr = linregress(x, y)
        lat15_i_slopes.append(slope)
        lat15_i_data_num.append(len(x))

    df_15_w_flux['direction'][i+1] = 270
    df_15_w_flux['0-300'][i+1] = lat15_i_slopes[0]
    df_15_w_flux['0-50'][i+1] = lat15_i_slopes[1]
    df_15_w_flux['50-100'][i+1] = lat15_i_slopes[2]
    df_15_w_flux['100-150'][i+1] = lat15_i_slopes[3]
    df_15_w_flux['150-200'][i+1] = lat15_i_slopes[4]
    df_15_w_flux['200-250'][i+1] = lat15_i_slopes[5]
    df_15_w_flux['250-300'][i+1] = lat15_i_slopes[6]

## Annual Slopes (Southwest)

In [ ]:
#Create empty dataframe with column names
df_15_sw_flux = pd.DataFrame(np.empty((5,8)), columns = depth_columns, index = season_rows)
df_10_sw_flux = pd.DataFrame(np.empty((5,8)), columns = depth_columns, index = season_rows)

In [ ]:
#Calculate vector sums and angles for 10 degree band
for i in range(0, 5):
    df_10_sw_flux['0-300'][i] = (df_10_s_flux['0-300'][i]**2 + df_10_w_flux['0-300'][i]**2)**(1/2)
    df_10_sw_flux['direction'][i] = math.degrees(np.arctan(df_10_w_flux['0-300'][i] / df_10_s_flux['0-300'][i])) + 180
    df_10_sw_flux['0-50'][i] = (df_10_s_flux['0-50'][i]**2 + df_10_w_flux['0-50'][i]**2)**(1/2)
    df_10_sw_flux['50-100'][i] = (df_10_s_flux['50-100'][i]**2 + df_10_w_flux['50-100'][i]**2)**(1/2)
    df_10_sw_flux['100-150'][i] = (df_10_s_flux['100-150'][i]**2 + df_10_w_flux['100-150'][i]**2)**(1/2)
    df_10_sw_flux['150-200'][i] = (df_10_s_flux['150-200'][i]**2 + df_10_w_flux['150-200'][i]**2)**(1/2)
    df_10_sw_flux['200-250'][i] = (df_10_s_flux['200-250'][i]**2 + df_10_w_flux['200-250'][i]**2)**(1/2)
    df_10_sw_flux['250-300'][i] = (df_10_s_flux['250-300'][i]**2 + df_10_w_flux['250-300'][i]**2)**(1/2)

In [ ]:
#Calculate vector sums for 15 degree band
for i in range(0, 5):
    df_15_sw_flux['0-300'][i] = (df_15_s_flux['0-300'][i]**2 + df_15_w_flux['0-300'][i]**2)**(1/2)
    
    df_15_sw_flux['direction'][i] = math.degrees(np.arctan(df_15_w_flux['0-300'][i] / df_15_s_flux['0-300'][i])) + 180
    
    df_15_sw_flux['0-50'][i] = (df_15_s_flux['0-50'][i]**2 + df_15_w_flux['0-50'][i]**2)**(1/2)
    df_15_sw_flux['50-100'][i] = (df_15_s_flux['50-100'][i]**2 + df_15_w_flux['50-100'][i]**2)**(1/2)
    df_15_sw_flux['100-150'][i] = (df_15_s_flux['100-150'][i]**2 + df_15_w_flux['100-150'][i]**2)**(1/2)
    df_15_sw_flux['150-200'][i] = (df_15_s_flux['150-200'][i]**2 + df_15_w_flux['150-200'][i]**2)**(1/2)
    df_15_sw_flux['200-250'][i] = (df_15_s_flux['200-250'][i]**2 + df_15_w_flux['200-250'][i]**2)**(1/2)
    df_15_sw_flux['250-300'][i] = (df_15_s_flux['250-300'][i]**2 + df_15_w_flux['250-300'][i]**2)**(1/2)

## Dataframes for 10 Degree Band (Slope)

In [ ]:
df_10_s_flux

In [ ]:
df_10_w_flux

In [ ]:
df_10_sw_flux

## Dataframes for 15 Degree Band (Slope)

In [ ]:
df_15_s_flux

In [ ]:
df_15_w_flux

In [ ]:
df_15_sw_flux

# 4. Convert values to Sverdrups <a name="convert_to_sv"></a>

### Steps:
1. m * (1 degree latitude / 111,111m) = dc/dy (units of 1/m)
2. dc/dy * Kh = 1D flux (units of m/s)
3. 1D flux * depth * width = 3D flux (units of m^3/s)
4. 3D flux / 1,000,000 = 3D flux (units of Sv = 1,000,000 m^3/s)

In [ ]:
depth = 300
width = 437157
lat_conv = 111111
Kh = 1000
sv_conv = 1000000

m_to_sv_factor = (Kh * width * depth) / (lat_conv * sv_conv)
print(f"To convert slopes to flow rates (using Kh = 1,000 m^2/s), multiply slope by {m_to_sv_factor}")

In [ ]:
df_10_s_flux

In [ ]:
depth_slabs = list(df_10_s_flux.columns)[1:]

for slab in depth_slabs:
    df_10_s_flux[slab] = df_10_s_flux[slab].apply(lambda x: x * m_to_sv_factor)
    df_10_s_sv = df_10_s_flux
    
    df_10_w_flux[slab] = df_10_w_flux[slab].apply(lambda x: x * m_to_sv_factor)
    df_10_w_sv = df_10_w_flux
    
    df_10_sw_flux[slab] = df_10_sw_flux[slab].apply(lambda x: x * m_to_sv_factor)
    df_10_sw_sv = df_10_sw_flux

    df_15_s_flux[slab] = df_15_s_flux[slab].apply(lambda x: x * m_to_sv_factor)
    df_15_s_sv = df_15_s_flux

    df_15_w_flux[slab] = df_15_w_flux[slab].apply(lambda x: x * m_to_sv_factor)
    df_15_w_sv = df_15_w_flux

    df_15_sw_flux[slab] = df_15_sw_flux[slab].apply(lambda x: x * m_to_sv_factor)
    df_15_sw_sv = df_15_sw_flux


# Dataframes for 10 Degree Band (Sv)

In [ ]:
print("Southern Freshwater Flow Rate for 10 Degree Band:")
df_10_s_sv

In [ ]:
print("Western Freshwater Flow Rate for 10 Degree Band:")
df_10_w_sv

In [ ]:
print("Southwestern Freshwater Flow Rate for 10 Degree Band:")
df_10_sw_sv

# Dataframes for 15 Degree Band (Sv)

In [ ]:
print("Southern Freshwater Flow Rate for 15 Degree Band:")
df_15_s_sv

In [ ]:
print("Western Freshwater Flow Rate for 15 Degree Band:")
df_15_w_sv

In [ ]:
print("Southwestern Freshwater Flow Rate for 15 Degree Band:")
df_15_sw_sv

# 5. Figure 3 (Scatter/line for both 10/15 Degree Bands) <a name="figure_3"></a>

In [ ]:
#Create list of x-coordinates for plotting
points_0 = np.array([0] * 7)
points_1 = np.array([1] * 7)
points_2 = np.array([2] * 7)
points_3 = np.array([3] * 7)
points_4 = np.array([4] * 7)

#Create list of y-coordinates for 10 degree band
annual_10 = np.array(df_10_sw_flux.loc['annual'])[1:]
fall_10 = np.array(df_10_sw_flux.loc['fall'])[1:]
winter_10 = np.array(df_10_sw_flux.loc['winter'])[1:]
spring_10 = np.array(df_10_sw_flux.loc['spring'])[1:]
summer_10 = np.array(df_10_sw_flux.loc['summer'])[1:]

#Create list of y-coordinates for 15 degree band
annual_15 = np.array(df_15_sw_flux.loc['annual'])[1:]
fall_15 = np.array(df_15_sw_flux.loc['fall'])[1:]
winter_15 = np.array(df_15_sw_flux.loc['winter'])[1:]
spring_15 = np.array(df_15_sw_flux.loc['spring'])[1:]
summer_15 = np.array(df_15_sw_flux.loc['summer'])[1:]

In [ ]:
#Figure 3 for 10 degree band (scatterplot)
plt.scatter(points_0, annual_10)
plt.scatter(points_1, fall_10)
plt.scatter(points_2, winter_10)
plt.scatter(points_3, spring_10)
plt.scatter(points_4, summer_10)
plt.xlabel('Season')
plt.ylabel('Freshwater Flux (Sv)')
plt.title('Southwestern Freshwater Flux for 10 Degree N Band')
print("0 : Annual Average\n1 : fall\n2 : winter\n3 : spring\n4 : summer ")

In [ ]:
#Figure 3 for 15 degree band (connected lines)
x_coord = np.arange(5)
for i in range(0, 7):
    annual = annual_10[i]
    fall = fall_10[i]
    winter = winter_10[i]
    spring = spring_10[i]
    summer = summer_10[i]
    slab = (annual, fall, winter, spring, summer)
    plt.plot(x_coord, slab)
plt.xlabel('Season')
plt.ylabel('Freshwater Flux (Sv)')
plt.title('Southwestern Freshwater Flux for 15 Degree N Band')
plt.gca().legend(('0-300m', '0-50m', '50-100m', '100-150m', '150-200m', '200-250m', '250-300m'))
print("Key:\n0 : Annual Average\n1 : fall\n2 : winter\n3 : spring\n4 : summer ")

In [ ]:
#Figure 3 for 15 degree band (scatterplot)
plt.scatter(points_0, annual_15)
plt.scatter(points_1, fall_15)
plt.scatter(points_2, winter_15)
plt.scatter(points_3, spring_15)
plt.scatter(points_4, summer_15)
plt.xlabel('Season')
plt.ylabel('Freshwater Flux (Sv)')
plt.title('Southwestern Freshwater Flux for 15 Degree N Band')
print("Key:\n0 : Annual Average\n1 : fall\n2 : winter\n3 : spring\n4 : summer ")

In [ ]:
#Figure 3 for 15 degree band (connected lines)
for i in range(0, 7):
    annual = annual_15[i]
    fall = fall_15[i]
    winter = winter_15[i]
    spring = spring_15[i]
    summer = summer_15[i]
    slab = (annual, fall, winter, spring, summer)
    plt.plot(x_coord, slab)
plt.xlabel('Season')
plt.ylabel('Freshwater Flux (Sv)')
plt.title('Southwestern Freshwater Flux for 15 Degree N Band')
plt.gca().legend(('0-300m', '0-50m', '50-100m', '100-150m', '150-200m', '200-250m', '250-300m'))
print("Key:\n0 : Annual Average\n1 : fall\n2 : winter\n3 : spring\n4 : summer ")

# 6. Multiple linear regression <a name="mlr"></a>

In [ ]:
#Multiple linear regression -> predicting freshwater values from latitude and longitude
X = lat_lon
y = fw

In [ ]:
#Split data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18)

#Train model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

#Test model
y_pred_train = regr.predict(X_train)
y_pred_test = regr.predict(X_test)

#Error analysis for testing, print result
mse_train = metrics.mean_squared_error(y_pred_train, y_train)
print(f"Mean Squared Error for Training Data: {mse_train}")

#Error analysis for testing, print result
mse_test = metrics.mean_squared_error(y_pred_test, y_test)
print(f"Mean Squared Error for Testing Data: {mse_test}")

In [ ]:
#Attempting to visualize multiple linear regression

#Set axes for 3D plots
x = X['latitude']
y = X['longitude']
z = y

#Set ranges for x and y for meshgrid
x_pred = np.linspace(3, 21, 30)   # range of porosity values
y_pred = np.linspace(84, 92, 30)  # range of brittleness values
xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
model_viz = np.array([xx_pred.flatten(), yy_pred.flatten()]).T

#Train model
ols = linear_model.LinearRegression()
model = ols.fit(X, y)
predicted = model.predict(model_viz)

#Collect coefficients and intercepts
coeff = model.coef_
lat_coeff = coeff[0]
lon_coeff = coeff[1]
intercept = model.intercept_

#Evaluate
r2 = model.score(X, y)

#Create 3D plot
plt.style.use('default')

fig = plt.figure(figsize=(12, 4))

ax1 = fig.add_subplot(131, projection='3d')
ax2 = fig.add_subplot(132, projection='3d')
ax3 = fig.add_subplot(133, projection='3d')

axes = [ax1, ax2, ax3]

for ax in axes:
    ax.plot(x, y, z, color='k', zorder=15, linestyle='none', marker='o', alpha=0.5)
    ax.scatter(xx_pred.flatten(), yy_pred.flatten(), predicted, facecolor=(0,0,0,0), s=20, edgecolor='#70b3f0')
    ax.set_xlabel('Latitude (Degrees N)', fontsize=12)
    ax.set_ylabel('Longitude (Degrees E)', fontsize=12)
    ax.set_zlabel('Freshwater (ppt)', fontsize=12)
    ax.locator_params(nbins=4, axis='x')
    ax.locator_params(nbins=5, axis='x')

ax1.text2D(0.2, 0.32, 'aegis4048.github.io', fontsize=13, ha='center', va='center',
           transform=ax1.transAxes, color='grey', alpha=0.5)
ax2.text2D(0.3, 0.42, 'aegis4048.github.io', fontsize=13, ha='center', va='center',
           transform=ax2.transAxes, color='grey', alpha=0.5)
ax3.text2D(0.85, 0.85, 'aegis4048.github.io', fontsize=13, ha='center', va='center',
           transform=ax3.transAxes, color='grey', alpha=0.5)

ax1.view_init(elev=28, azim=120)
ax2.view_init(elev=4, azim=114)
ax3.view_init(elev=60, azim=165)

fig.suptitle('$R^2 = %.2f$' % r2, fontsize=20)

fig.tight_layout()

## Formulas for multiple linear regression

<h1><center>$\hat{Y} = \hat{\beta}_{0} + \sum \limits _{j=1} ^{p} X_{j}\hat{\beta}_{j} $<h1><center>

<h1><center>$\hat{Y} = \hat{\beta}_{0} + X_{1}\hat{\beta}_{1} + X_{2}\hat{\beta}_{2}$<h1><center>

$\hat{Y}$ = Predicted Freshwater Value (ppt)

$\hat{\beta}_{0}$ = Intercept

$X_{1}$ = Latitude Data (Degrees N)

$\hat{\beta}_{1}$ = Latitude Coefficient

$X_{2}$ = Longitude Data (Degrees E)

$\hat{\beta}_{2}$ = Longitude Coefficient

In [ ]:
print(f"Intercept: {intercept}")
print(f"Latitude Coefficient: {lat_coeff}")
print(f"Longitude Coefficient: {lon_coeff}")

# 7. Various Horizontal Mixing Coefficients <a name="various_kh"></a>

In [ ]:
#Make list of rows and columns
band_direction_idx = ['15 degree band SW', '15 degree band S', '15 degree band W', 
                      '10 degree band SW', '10 degree band S', '10 degree band W']
depth_columns = ['0-300', '0-50', '50-100', '100-150', '150-200', '200-250', '250-300']

#Create consolidated dataframe for annual values
df_kh = pd.DataFrame(np.empty((6,7)), columns = depth_columns, index = band_direction_idx)
df_kh.loc['15 degree band SW'] = df_15_sw_sv.loc['annual']
df_kh.loc['15 degree band S'] = df_15_s_sv.loc['annual']
df_kh.loc['15 degree band W'] = df_15_w_sv.loc['annual']
df_kh.loc['10 degree band SW'] = df_10_sw_sv.loc['annual']
df_kh.loc['10 degree band S'] = df_10_s_sv.loc['annual']
df_kh.loc['10 degree band W'] = df_10_w_sv.loc['annual']

#Set Kh value
Kh = 1000
print(f"Horizontal mixing coefficient Kh = {Kh}")
df_kh

In [ ]:
#Create dataframe for Kh = 2000
new_Kh = 2000
Kh_ratio = new_Kh / Kh

print(f"Horizontal mixing coefficient Kh = {new_Kh}")
df_kh_2000 = df_kh * Kh_ratio
df_kh_2000

In [ ]:
#Create dataframe for Kh = 5000
new_Kh = 5000
Kh_ratio = new_Kh / Kh

print(f"Horizontal mixing coefficient Kh = {new_Kh}")
df_kh_5000 = df_kh * Kh_ratio
df_kh_5000